## CONFIGURE PRIMARY CLUSTER

In [ ]:
%%bash
gcloud auth login
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_primary")


vault write -f sys/replication/dr/primary/enable primary_cluster_addr=$(terraform output -raw cluster_primary_fqdn_8201)
sleep 5

vault write -format=json sys/replication/dr/primary/secondary-token id=dr12 | jq -r .wrap_info.token > dr_token.txt



## CONFIGURE SECONDARY CLUSTER

In [ ]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_dr_fqdn_8200)
vault_token_dr=$(terraform output -raw cluster_dr_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_dr")

vault write sys/replication/dr/secondary/enable  \
    primary_api_addr=$(terraform output -raw cluster_primary_fqdn_8200) \
    token=$(cat dr_token.txt)

In [ ]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault policy write dr-secondary-promotion - <<EOF
path "sys/replication/dr/secondary/promote" {
  capabilities = [ "update" ]
}

# To update the primary to connect
path "sys/replication/dr/secondary/update-primary" {
    capabilities = [ "update" ]
}

# Only if using integrated storage (raft) as the storage backend
# To read the current autopilot status
path "sys/storage/raft/autopilot/state" {
    capabilities = [ "update" , "read" ]
}

path "sys/storage/raft/*" {
    capabilities = [ "update" , "read", "create", "delete", "patch", "sudo" ]
}
EOF

vault write auth/token/roles/failover-handler \
    allowed_policies=dr-secondary-promotion \
    orphan=true \
    renewable=false \
    token_type=batch


vault token create -role=failover-handler -ttl=8h

vault read sys/storage/raft/autopilot/state

In [ ]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault token create -role=failover-handler -ttl=8h -format=json | jq -r .auth.client_token > dr_batch_token.txt

In [ ]:
%%bash
export VAULT_ADDR=$(terraform output -raw cluster_dr_fqdn_8200)
export VAULT_TOKEN=$(cat dr_batch_token.txt)

vault read sys/storage/raft/autopilot/state